# 文件中保存模型结构和权重参数

In [23]:
import numpy as np
import torch.optim as optim
import torch.nn as nn

In [24]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) 
t_u = torch.tensor(t_u).unsqueeze(1) 

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val
t_un_val

tensor([[5.6300],
        [6.0400]])

In [25]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [26]:
seq_model = nn.Sequential(
            nn.Linear(1, 13), 
            nn.Tanh(),
            nn.Linear(13, 1)) # 这里张量的输入必须和前面输出相同
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [27]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))

Epoch 1, Training loss 204.6776, Validation loss 153.8250
Epoch 1000, Training loss 2.5927, Validation loss 9.1336
Epoch 2000, Training loss 1.3800, Validation loss 7.8509
Epoch 3000, Training loss 1.2893, Validation loss 7.4323
Epoch 4000, Training loss 1.2541, Validation loss 7.4136
Epoch 5000, Training loss 1.2258, Validation loss 7.4807
output tensor([[13.1765],
        [16.1987]], grad_fn=<AddmmBackward0>)


In [28]:
t_u_train.shape

torch.Size([9, 1])

In [29]:
t_u_train = t_u_train[1,:]

In [30]:
import torch
torch_model = seq_model # pytorch模型加载
 
# set the model to inference mode
torch_model.eval()
 
x = torch.tensor(t_u_train, dtype=torch.float32) # 生成张量
export_onnx_file = "test.onnx" # 目的ONNX文件名
torch.onnx.export(torch_model,
                    x,
                    export_onnx_file,
                    opset_version=10,
                    do_constant_folding=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1428\3604413986.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(t_u_train, dtype=torch.float32) # 生成张量
